In [ ]:
# importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
# For Transfer Learning
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizer_v2.adam import Adam
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.utils.np_utils import to_categorical
# Keras library for CIFAR-10 dataset
from keras.datasets import cifar10

# **Load the dataset**

In [ ]:
# Downloading the CIFAR dataset
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [ ]:
# One Hot Encoding
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)
# Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((50000, 32, 32, 3), (50000, 10))
((12500, 32, 32, 3), (12500, 10, 2))
((10000, 32, 32, 3), (10000, 10))


As you can see we have successfully converted the target variable into a one-hot encoding representation. Now let’s use the ImageDataGenerator module to create the batches of the images so that while training we can feed data sequentially. Below, we will initiate the separate data generator function for each phase, that is training, testing, and validation. 

In [ ]:
# Image Data Augmentation
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=.1)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True, zoom_range=.1)
test_generator = ImageDataGenerator()

In [ ]:
# Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

# **Loading and training VGG19**

Now we will start with building the model, first, we will initialize the rained model that VGG-19 with pre-trained weights from the ImageNet competition.

In [ ]:
# Defining the VGG Convolutional Neural Net
base_model = VGG19(include_top = False, weights = 'imagenet',
                input_shape = (32,32,3), classes = y_train.shape[1])

Above is the base model that we have initialized, now we will add the last classifier model on top of the above model. Below we will first create Keras sequential model and will first add the base model that is the VGG-19 model and latter will add 4 dense layers with activation function as ReLU and the last layer will be the classifier layer with 10 units and softmax as the activation function.

In [ ]:
# Adding the final layers to the above base models where the actual classification is done in the dense layers
model= Sequential()
model.add(base_model)
model.add(Flatten())
#Adding the Dense layers along with activation and batch normalization
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu')))
model.add(Dense(256,activation=('relu')))
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
model.add(Dropout(.2))
model.add(Dense(10,activation=('softmax')))

In [ ]:
# Checking the final model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 1024)              525312    
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

As we can see there are a total of 21240010 parameters to be trained. Now we will initialize the hyperparameter of the model such as batch size, epochs, and learning rate.

In [ ]:
# Initializing the hyperparameters
batch_size= 100
epochs=50
learn_rate=.001

In [ ]:
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['acc'])

Below we will start the training model, the model will train for the 50 epochs and for each epoch 375 samples will be utilized.  

# **Training the model**

In [ ]:
# Training the model
history = model.fit_generator(train_generator.flow(x_train, y_train, batch_size= 100), epochs = epochs,
                 steps_per_epoch = x_train.shape[0]//batch_size,
                 validation_data = val_generator.flow(x_val, y_val, batch_size = 100))

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


  1/500 [..............................] - ETA: 1:35:41 - loss: 2.3001 - acc: 0.1600

KeyboardInterrupt: ignored

# **Evaluating the model**

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'],color='b',label='Training Loss')
plt.plot(history.history['val_loss'],color='r',label='Validation Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['acc'],color='b',label='Training Loss')
plt.plot(history.history['val_acc'],color='r',label='Validation Loss')
plt.legend()

In [ ]:
# Making prediction
y_pred=model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)
y_true=np.argmax(y_test,axis=1)

Below we are defining the function which will plot the confusion metrics.

In [ ]:
# Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                       normalize=False,
                       title=None,
                       cmap=plt.cm.Blues):
if not title:
     if normalize:
         title = 'Normalized confusion matrix'
     else:
         title = 'Confusion matrix, without normalization'
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
if normalize:
     cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
     print("Normalized confusion matrix")
else:
     print('Confusion matrix, without normalization')
# print (confusion matrix)
fig, ax = plt.subplots(figsize=(7,7))
im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
ax.figure.colorbar(im, ax=ax)
# We want to show all ticks...
ax.set(xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes, yticklabels=classes,
        title=title,
        ylabel='True label',
        xlabel='Predicted label')
#Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
          rotation_mode="anchor")
# Loop over data dimensions and create text annotations.
fmt = '.2f' if normalize else 'd'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
     for j in range(cm.shape[1]):
         ax.text(j, i, format(cm[i, j], fmt),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")
fig.tight_layout()

Now after defining the function First, we will see the exact number of correct and incorrect classifications using the non-normalized confusion matrix and then we will see the same percentage using the normalized confusion matrix. 

In [ ]:
np.set_printoptions(precision=2)
# Plotting the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred)
# Defining the class labels
class_names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Plotting non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes = class_names, title='Confusion matrix, without normalization')

In [ ]:
# Plotting normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes = class_names, normalize = True, title = 'Normalized confusion matrix')

In [ ]:
# Classification report
from sklearn.metrics import classification_report
print((classification_report(y_pred, np.argmax(y_test,axis=1))))